In [15]:
import pandas as pd

# Replace with your actual test file path
fold = "1"  # Change as needed
test_file = f"folds/fold_{fold}_test.csv"
test_data = pd.read_csv(test_file)

# Identify label columns (slider-based labels)
label_columns = [col for col in test_data.columns if "slider" in col]
feature_columns = [col for col in test_data.columns if col not in label_columns + ["Timestamp", "participant_id"]]

# Extract features and true labels
X_test = test_data[feature_columns]


In [16]:
import pickle

# Load the Random Forest model for a specific label
label_name = label_columns[0]  # Change if using multiple labels
rf_model_path = f"folds_original_models/fold_{fold}/models/{label_name}/random_forest/random_forest_{label_name}.pkl"

with open(rf_model_path, "rb") as f:
    rf_model = pickle.load(f)

# Generate Predictions
y_test = test_data[label_name]  # Ground truth labels
y_pred = rf_model.predict(X_test)


In [24]:
import pickle

lstm_model_path = f"folds_original_models/fold_{fold}/models/{label_name}/lstm/lstm_model_{label_name}.h5"
lstm_model = load_model(lstm_model_path)

# Reshape data for LSTM (assuming 1D input)
X_test_lstm = np.expand_dims(X_test, axis=2)

# Generate Predictions
y_probs_lstm = lstm_model.predict(X_test_lstm)
y_pred_lstm = y_probs_lstm.argmax(axis=1)


34/34 [==============================] - 2s 35ms/step


In [17]:
from tensorflow.keras.models import load_model
import numpy as np

# Load CNN model
cnn_model_path = f"folds_original_models/fold_{fold}/models/{label_name}/cnn/cnn_model_{label_name}.h5"
cnn_model = load_model(cnn_model_path)

# Reshape data for CNN (assuming 1D input)
X_test_cnn = np.expand_dims(X_test, axis=2)

# Generate Predictions
y_probs_cnn = cnn_model.predict(X_test_cnn)
y_pred_cnn = y_probs_cnn.argmax(axis=1)


2025-02-26 21:26:46.049768: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 21:26:46.077314: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-26 21:26:46.077335: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 21:26:46.077375: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 21:26:46.084907: I tensorflow/core/platform/cpu_feature_g

34/34 [==============================] - 0s 3ms/step


In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, model_name, label_name):
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=set(y_true), yticklabels=set(y_true))
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"Confusion Matrix for {model_name} ({label_name})")
    plt.show()

# Example: Plot for Random Forest
plot_confusion_matrix(y_test, y_pred, "Random Forest", label_name)

# Example: Plot for CNN
plot_confusion_matrix(y_test, y_pred_cnn, "CNN", label_name)


ModuleNotFoundError: No module named 'seaborn'

In [25]:
import pandas as pd

# Save Random Forest predictions
rf_predictions_df = pd.DataFrame({"y_true": y_test, "y_pred": y_pred})
rf_predictions_df.to_csv("rf_predictions.csv", index=False)

# Save CNN predictions (if applicable)
cnn_predictions_df = pd.DataFrame({"y_true": y_test, "y_pred": y_pred_cnn})
cnn_predictions_df.to_csv("cnn_predictions.csv", index=False)

# Save LSTM predictions (if applicable)
lstm_predictions_df = pd.DataFrame({"y_true": y_test, "y_pred": y_pred_lstm})
lstm_predictions_df.to_csv("lstm_predictions.csv", index=False)

print("Saved predictions as CSV files.")


Saved predictions as CSV files.
